In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importamos las librerias que vamos a usar

In [46]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np
import pandas as pd

# Libreria para graficos
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

#libreria para dividir los datos para entrenamiento y prueba
from sklearn.model_selection import train_test_split

#libreria para transformar textos a numeros
from sklearn.preprocessing import LabelEncoder

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

Cargamos el dataset y procesamos algunos datos

In [33]:
data = pd.read_csv('/content/drive/MyDrive/SIS420 IA 1/datasets/Propios/star_classification.csv', delimiter=',')
#display(data)
data = data.drop(['obj_ID','rerun_ID'], axis = 1 ) #borramos la columna obj_ID porque tiene ID's unicos y rerun_ID que tiene un solo valor en todas las filas
display(data)

,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,2,581,1.055431e+19,GALAXY,0.000000,9374,57749,438
99996,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,1,289,8.586351e+18,GALAXY,0.404895,7626,56934,866
99997,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,4,308,3.112008e+18,GALAXY,0.143366,2764,54535,74
99998,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,4,131,7.601080e+18,GALAXY,0.455040,6751,56368,470


# 1.- Insertamos los nuevos atributos que contengan: caracteres, texto y valores booleanos, que esten relacionados a la temática del dataset.

In [35]:
#"variable_brightness" atributo que indica si el objeto presenta una variabilidad en su brillo.
#Las estrellas tienden a tener variaciones más pronunciadas, mientras que las galaxias y cuásares pueden tener brillo más constante.
data_sintetico = data.copy()
data_sintetico['variable_brightness'] = np.random.choice([True, False], size=len(data_sintetico))

#"nuclear_activity" atributo que indica el tipo de actividad nuclear. Los cuásares suelen tener una actividad intensa, las galaxias menos,
#y las estrellas no tienen actividad nuclear central.'I' para actividad intensa, 'M' para moderada, 'N' para nada
valores_nuclear_activity = ['I', 'M', 'N']
data_sintetico['nuclear_activity'] = np.random.choice(valores_nuclear_activity, size=len(data_sintetico))


#"distance_description" este atributo de texto describe la distancia relativa del objeto, usando el desplazamiento al rojo (redshift)
# para determinar si es una estrella cercana, una galaxia distante o un cuásar muy distante.

descripciones_distancia = [
    'Very close object',
    'Moderately distant object',  # Define posibles descripciones para la columna de texto
    'Far object',
    'Extremely distant object'
]
data_sintetico['distance_description'] = np.random.choice(descripciones_distancia, size=len(data_sintetico)) #Asignar aleatoriamente descripciones a la nueva columna

#display(data_sintetico)
print(data_sintetico.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   alpha                 100000 non-null  float64
 1   delta                 100000 non-null  float64
 2   u                     100000 non-null  float64
 3   g                     100000 non-null  float64
 4   r                     100000 non-null  float64
 5   i                     100000 non-null  float64
 6   z                     100000 non-null  float64
 7   run_ID                100000 non-null  int64  
 8   cam_col               100000 non-null  int64  
 9   field_ID              100000 non-null  int64  
 10  spec_obj_ID           100000 non-null  float64
 11  class                 100000 non-null  object 
 12  redshift              100000 non-null  float64
 13  plate                 100000 non-null  int64  
 14  MJD                   100000 non-null  int64  
 15  f

Convertimos las columnas que contienen texto o letras a números

In [38]:
#convertimos la columna con valores Booleanos a numeros
data_sintetico['variable_brightness'] = data_sintetico['variable_brightness'].astype(int)

#selecciona las columnas del dataset de tipo object
columnas_categoricas = data_sintetico.select_dtypes(include=['object']).columns

for columna in columnas_categoricas: #Recorremos cada columna categorica
  #El LabelEncoder se utiliza para codificar las etiquetas categóricas como valores numéricos
  le = LabelEncoder() #creamos una instancia de LabelEncoder()
  data_sintetico[columna] = le.fit_transform(data_sintetico[columna])  #Su método fit_transform ajusta y trasnforma los datos de la columna

display(data_sintetico)

,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID,variable_brightness,nuclear_activity,distance_description
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,2,79,6.543777e+18,0,0.634794,5812,56354,171,1,1,0
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,5,119,1.176014e+19,0,0.779136,10445,58158,427,1,0,1
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,2,120,5.152200e+18,0,0.644195,4576,55592,299,0,1,2
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,3,214,1.030107e+19,0,0.932346,9149,58039,775,0,1,0
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,3,137,6.891865e+18,0,0.116123,6121,56187,842,1,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,2,581,1.055431e+19,0,0.000000,9374,57749,438,0,1,2
99996,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,1,289,8.586351e+18,0,0.404895,7626,56934,866,0,0,3
99997,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,4,308,3.112008e+18,0,0.143366,2764,54535,74,1,2,1
99998,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,4,131,7.601080e+18,0,0.455040,6751,56368,470,1,0,3


# 2.- Duplicamos los ejemplos del dataset considerando rangos y criterios que garanticen la consistencia del dataset.

Esto lo hare aplicando una variacion aleatoria controlada a los valores ya existentes, la variacion sera del 10%

In [44]:
porcentaje_variacion = 0.10  # Defino el porcentaje de variación

# Función para aplicar variaciones a los valores numéricos de tipo float64
def aplicar_variacion(valor, es_float):
    if es_float:
        # Generar un factor de variación aleatorio
        factor = np.random.uniform(1 - porcentaje_variacion, 1 + porcentaje_variacion)
        return valor * factor
    return valor

# Función para generar los nuevos valores
def generar_datos_modificados(df, porcentaje_variacion):
    datos_modificados = []

    # Identificar las columnas de tipo float64
    columnas_float64 = df.select_dtypes(include=[np.float64]).columns

    for index, fila in df.iterrows():
        # Agregar la fila original
        datos_modificados.append(fila)

        # Crear una nueva fila con valores alterados solo en columnas de tipo float64
        fila_modificada = fila.copy()

        for col in fila_modificada.index:
            es_float = col in columnas_float64
            fila_modificada[col] = aplicar_variacion(fila_modificada[col], es_float)

        # Agregar la fila modificada
        datos_modificados.append(fila_modificada)

    # Convertir el resultado en el formato DataFrame
    return pd.DataFrame(datos_modificados)

# Generar el DataFrame final en base a los datos ya existentes
data_sintetico_duplicado = generar_datos_modificados(data_sintetico, porcentaje_variacion)

display(data_sintetico_duplicado)

,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID,variable_brightness,nuclear_activity,distance_description
0,135.689107,32.494632,23.878820,22.275300,20.395010,19.165730,18.793710,3606.0,2.0,79.0,6.543777e+18,0.0,0.634794,5812.0,56354.0,171.0,1.0,1.0,0.0
0,146.765308,30.151727,25.440407,21.459480,19.672412,19.120436,17.337247,3606.0,2.0,79.0,6.583642e+18,0.0,0.619704,5812.0,56354.0,171.0,1.0,1.0,0.0
1,144.826101,31.274185,24.777590,22.831880,22.584440,21.168120,21.614270,4518.0,5.0,119.0,1.176014e+19,0.0,0.779136,10445.0,58158.0,427.0,1.0,0.0,1.0
1,153.422369,31.810060,22.441891,23.276768,21.655407,22.418778,20.360170,4518.0,5.0,119.0,1.287985e+19,0.0,0.840882,10445.0,58158.0,427.0,1.0,0.0,1.0
2,142.188790,35.582444,25.263070,22.663890,20.609760,19.348570,18.948270,3606.0,2.0,120.0,5.152200e+18,0.0,0.644195,4576.0,55592.0,299.0,0.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99997,225.683342,14.735267,19.342072,19.341773,17.533060,17.995037,16.610544,5314.0,4.0,308.0,3.077937e+18,0.0,0.132279,2764.0,54535.0,74.0,1.0,2.0,1.0
99998,212.268621,46.660365,25.350390,21.637570,19.913860,19.072540,18.624820,3650.0,4.0,131.0,7.601080e+18,0.0,0.455040,6751.0,56368.0,470.0,1.0,0.0,3.0
99998,220.029207,45.207152,23.778891,20.125934,21.726028,19.165292,16.915177,3650.0,4.0,131.0,7.667561e+18,0.0,0.491387,6751.0,56368.0,470.0,1.0,0.0,3.0
99999,196.896053,49.464643,22.621710,21.797450,20.601150,20.009590,19.280750,3650.0,4.0,60.0,8.343152e+18,0.0,0.542944,7410.0,57104.0,851.0,1.0,0.0,1.0


# 3.-Crea una grafica representativa de los datos del dataset inicial y otra con los datos del dataset sintético generado.

funcion para graficar

In [47]:
def graficar_regresion_logistica(df, titulo):
    plt.figure(figsize=(10, 6))

    # Seleccionar las columnas para X y y
    X = df[['u', 'g']]
    y = df['class']

    # Escalar los datos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Ajustar el modelo
    model = LogisticRegression()
    model.fit(X_scaled, y)

    # Crear una malla de puntos para graficar la frontera de decisión
    h = .02  # tamaño del paso en la malla
    x_min, x_max = X_scaled[:, 0].min() - 1, X_scaled[:, 0].max() + 1
    y_min, y_max = X_scaled[:, 1].min() - 1, X_scaled[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z, alpha=0.3, cmap='viridis')
    plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=y, edgecolor='k', cmap='viridis')

    plt.title(titulo)
    plt.xlabel('u')
    plt.ylabel('g')
    plt.grid(True)
    plt.show()


Graficamos los datos del dataset original

In [ ]:
graficar_regresion_logistica(data, 'Datos Iniciales con Frontera de Decisión')
